# SHOR with multiple errors, exponential decay



In [3]:
#QISKIT
import qiskit.quantum_info as qi

from qiskit import Aer
from qiskit import execute, transpile, assemble
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info.operators import Operator
from qiskit.visualization import plot_histogram
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector

from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.library import SaveDensityMatrix

#PYTHON
import numpy as np
from numpy import linalg, pi
from sympy import *
from scipy.sparse import csr_matrix

import time
import random
from random import sample
import pandas as pd
import math
from fractions import Fraction
import matplotlib.pyplot as plt
import pickle

#CHECK N AND y, a
def check(number):
    if number % 2 == 0:
        print("Even, 2 is a divisor")
        return
        
    if isprime(number) == True:
        print("Prime number")
        return
    
    factorization = factorint(number)
    for i in range(len(factorization.keys())):
        p = list(factorization)[i]
        q = list(factorization.values())[i]
        if isprime(p) and q != 1:
            print("Prime Power: %i^%i, %i is a divisor" % ( p, q, p))
            return
    
    y = random.randrange(2, number - 1)
    a = math.gcd(y, number)
    if a > 1:
        print("Divisor: a = %i" %(a))
        return 
        
    else:
        print("y = %i and N = %i are coprime - r well defined" %  (y, N))
        return 
    
#MODULAR EXPONENTIATION GATE
def mod_exp(n, y, N, power):
    dim = 2**n
    matrix_0 = np.zeros((dim, dim))
    
    #CREATE F(X)
    for i in range(N):
        j = (i*y)%N
        matrix_0[j][i]=1
             
    for ii in range(N, dim):
        matrix_0[ii][ii] = 1

    temp = matrix_0       
    #temp = csr_matrix(matrix_0)
    for iii in range(power):
        matrix = temp.dot(temp)
        #matrix = temp.multiply(temp)
        temp = matrix

    #CREATE GATE
    U = UnitaryGate(temp)
    U.name = "%i^%i mod %i" % ( y, 2**power, N)
    c_U = U.control()
    return  c_U

#ERROR ROUTINE
def errori(circuit, n):  
    epsilon = 0.5                                              
    extracted = random.uniform(0, 1)
    position = random.randint(0, n)
    
    if (extracted < epsilon):  
        circuit.x(position)   
    else:
        circuit.z(position)
    return circuit 

#IQFT                          
def qft_dagger(n, circuit, case, case_max): 
    random_list = []
    if case == 0:
        random_list = [0]
    for i in range(0, case):  
        random_list = random.sample(range(0, case_max), case)  
    
    pos = 0
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
        
    for j in range(n):
        for m in range(j):
            circuit.cp(-np.pi/float(2**(j-m)), m, j)
            pos += 1
            circuit.barrier()
            for jj in range(len(random_list)):
                if pos == random_list[jj]:
                    errori(circuit, n) #
                    circuit.barrier()
        circuit.barrier()
        
        circuit.h(j)
        pos += 1
        circuit.barrier()
        for jj in range(len(random_list)):
            if pos == random_list[jj]:
                errori(circuit, n) 
                circuit.barrier()
    return circuit

if __name__ == "__main__":
    print("Starting execution")

    start = time.time()

    N = 15
    r = 6
    y = 8
    case_max = 36
    case_step = 1
    repetitions = 1
    
    #coprimes = [2,4,7,8,11,13]
    n = math.ceil(math.log(N,2))
    n_count = 2*n
    
    #MATRICES FOR MODEXP
    matrices = []
    for q in range(2*n):
        matrices.append(mod_exp(n, y, N, q))   
    
    #DATA
    data = []
    f = open("15.txt", "w")
    f.write('case'+ '\t' + 'rep'+ '\t' + 'j'+ '\t' + 'r'+ '\t' + 'k'+ '\t' + 'guesses'+ '\n')
    print('case'+ '\t' + 'rep'+ '\t' + 'j'+ '\t' + 'r'+ '\t' + 'k'+ '\t' + 'guesses'+ '\n')
    
    #USEFUL Js
    useful_js = []
    for c in range(1, r):
        costant = c*2**(n_count)/r
        useful_js.append(math.ceil(costant))
        useful_js.append(math.floor(costant))
    useful_js_nodup = [*set(useful_js)]
    
    #MAIN
    for case in range(0, case_max, case_step):
        js = []
        fact = []

        for rep in range(repetitions):
            #y = coprimes[random.randint(0, 5)]
            position = random.randint(0, 2*n)
            
            #CIRCUIT
            control = QuantumRegister(2*n, 'r1')
            target  = QuantumRegister(n, 't1')
            classic = ClassicalRegister(2*n, 'c')
            circuit = QuantumCircuit(control, target, classic)
            circuit.h(range(2*n))
            circuit.x(2*n + n -1)
            for q in range(2*n):
                circuit.append(matrices[q], [q] + [i+2*n for i in range(n)])   
            circuit.barrier()
            qft_dagger(2*n, circuit, case, case_max) 
            circuit.barrier()
            circuit.measure(range(2*n), range(2*n))        

            #SIMULATION
            aer_sim = Aer.get_backend('aer_simulator')        
            job = aer_sim.run(transpile(circuit, aer_sim), shots=1, memory=True)
            readings = job.result().get_memory()

            #PHASES
            j = int(readings[0], 2)
            js.append(j)
            
            phase = int(readings[0],2)/(2**n_count)                     
            #CONTINUED FRACTIONS
            if phase != 0:
                frac = Fraction(phase).limit_denominator(N)
                r, k = frac.denominator, frac.numerator
                guesses = [gcd(y**(r//2)-1, N), gcd(y**(r//2)+1, N)]
                fact.append([r, k, guesses])
            if phase == 0:
                r, k = 0, 0  
                guesses = [0, 0]
            f.write( str(case)+ '\t'+ str(rep) +'\t'+ str(j)+ '\t'+ str(r)+ '\t'+ str(k) + '\t' + str(guesses) + '\n')
            print( case, '\t', rep,'\t', j, '\t', r,'\t', k,'\t' ,guesses )
        sum = 0
        for i in range(len(js)):
            for j in range(len(useful_js_nodup)):
                if js[i] == useful_js_nodup[j]:
                    sum = sum + 1

        data.append([case, sum])

    print(data)
    end = time.time()
    print(end - start)
    f.write('FINAL DATA:' + '  ' + str(data))
    f.close()
    


Starting execution
case	rep	j	r	k	guesses

0 	 0 	 192 	 4 	 3 	 [3, 5]
1 	 0 	 0 	 0 	 0 	 [0, 0]
2 	 0 	 0 	 0 	 0 	 [0, 0]
3 	 0 	 17 	 15 	 1 	 [1, 3]
4 	 0 	 200 	 9 	 7 	 [15, 1]
5 	 0 	 0 	 0 	 0 	 [0, 0]
6 	 0 	 72 	 7 	 2 	 [1, 3]
7 	 0 	 4 	 1 	 0 	 [15, 1]
8 	 0 	 213 	 6 	 5 	 [1, 3]
9 	 0 	 2 	 1 	 0 	 [15, 1]
10 	 0 	 160 	 8 	 5 	 [15, 1]
11 	 0 	 120 	 15 	 7 	 [1, 3]
12 	 0 	 146 	 7 	 4 	 [1, 3]
13 	 0 	 0 	 0 	 0 	 [0, 0]
14 	 0 	 218 	 13 	 11 	 [3, 5]
15 	 0 	 69 	 15 	 4 	 [1, 3]
16 	 0 	 226 	 9 	 8 	 [15, 1]
17 	 0 	 120 	 15 	 7 	 [1, 3]
18 	 0 	 54 	 14 	 3 	 [1, 3]
19 	 0 	 119 	 15 	 7 	 [1, 3]
20 	 0 	 165 	 14 	 9 	 [1, 3]
21 	 0 	 217 	 13 	 11 	 [3, 5]
22 	 0 	 176 	 13 	 9 	 [3, 5]
23 	 0 	 94 	 11 	 4 	 [1, 3]
24 	 0 	 104 	 5 	 2 	 [3, 5]
25 	 0 	 3 	 1 	 0 	 [15, 1]
26 	 0 	 182 	 7 	 5 	 [1, 3]
27 	 0 	 22 	 12 	 1 	 [3, 5]
28 	 0 	 103 	 5 	 2 	 [3, 5]
29 	 0 	 225 	 8 	 7 	 [15, 1]
30 	 0 	 68 	 15 	 4 	 [1, 3]
31 	 0 	 174 	 13 	 9 	 [3, 5]
32 	 